Inspiration/tutorial: https://www.youtube.com/watch?v=PyjBd7IDYZs

### Goal

Plug the USB camera into the double header microscope, press a button and have it count Ki-67 percentage.

Motivation: Current Ki-67 is 'imprecise'

Plan
- USB camera in double header, connected to laptop.
- Laptop reads camera feed with openCV
- Apply filters to create an image with only ki-67 positive cells and an image that only shows non-ki-67 cells
    - Filters are going to be using YOLOv2
- Apply a counter for each of the images
- Determine percentage
- Evaluate manually to determine if more effective than current standard of practice.

Software:
- Python 
- TensorFlow 
- openCV

Network
- Weights from YOLOv2 from:https://pjreddie.com/darknet/yolov2/
    - Note that yolov3 is available to try

# SETUP:

#### Create new environement
```pipenv --python 3.6``` (3.6.4)

#### Active environment
```pipenv shell```

#### Installations
```pipenv install opencv-python``` (3.4.2.16)

```pipenv install tensorflow``` (1.9.0)

```pipenv install ipykernel``` (4.8.2)

#### Create a new kernel for your environment
```ipython kernel install --user --name=vision_microscope```

```jupyter notebook``` --> then go 'kernel' -> 'change kernel' -> 'vision_microscope'


In [1]:
# Modules

import numpy as np
import cv2
import tensorflow as tf




# Hardware integration

In [3]:
# Determine what hardware is connected
!system_profiler SPUSBDataType

2018-07-22 21:07:26.579 system_profiler[4283:47504] SPUSBDevice: IOCreatePlugInInterfaceForService failed 0xe00002be
USB:

    USB 3.0 Bus:

      Host Controller Driver: AppleUSBXHCISPT
      PCI Device ID: 0xa12f 
      PCI Revision ID: 0x0031 
      PCI Vendor ID: 0x8086 

        iBridge:

          Product ID: 0x8600
          Vendor ID: 0x05ac (Apple Inc.)
          Version: 1.01
          Manufacturer: Apple Inc.
          Location ID: 0x14200000

    USB 3.1 Bus:

      Host Controller Driver: AppleUSBXHCIAR
      PCI Device ID: 0x15d4 
      PCI Revision ID: 0x0002 
      PCI Vendor ID: 0x8086 
      Bus Number: 0x01 

    USB 3.1 Bus:

      Host Controller Driver: AppleUSBXHCIAR
      PCI Device ID: 0x15d4 
      PCI Revision ID: 0x0002 
      PCI Vendor ID: 0x8086 
      Bus Number: 0x00 



In [8]:
usb_camera_id = 0 # zero, one, two, three, who knows. I like one
# usually 1 for USB cam, 0 for isight cam

In [11]:


cap = cv2.VideoCapture(usb_camera_id)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)    #BGR2GRAY for greyscale
    
    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

# Image manipulation with OpenCV

In [4]:
# load an image and modify with a mask
img = cv2.imread('kitten.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

lower_green = np.array([0,0, 150])
upper_green = np.array([255, 255, 255])

mask = cv2.inRange(hsv, lower_green, upper_green)
res = cv2.bitwise_and(img, img, mask=mask)

cv2.imshow('res1.png', res)
cv2.waitKey(0)

error: OpenCV(3.4.1) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:11214: error: (-215) (scn == 3 || scn == 4) && (depth == 0 || depth == 5) in function cvtColor
